In [1]:
import re

def getIngredientsAndAllergens(text):
    splitText = text.split('(contains')
    ingredientsSearch = re.findall('(([a-z]*)\s)', splitText[0])
    return {
        'ingredients' : [ingr[1] for ingr in ingredientsSearch],
        'allergens' : [a.strip() for a in splitText[1].replace(')','').split(',')]
        }

def transformDataTODict(data):
    res = []
    for rule in data:
        tempRule = {}
        for a in rule['allergens']:
            for ingredient in rule['ingredients']:
                temp = tempRule.get(a, [])
                if(not(ingredient in temp)): 
                    tempRule[a] = temp+[ingredient]
        res.append(tempRule)
    return res

In [2]:
fileInput = open('data.txt', mode='r')
dataInput = [getIngredientsAndAllergens(text) for text in fileInput.readlines()]
dataInput1 = transformDataTODict(dataInput)
listOfAlle = list(set([ a for rule in dataInput for a in rule['allergens']]))
fileInput.close()

In [3]:
def getIngredientsListForAllerg(aller, listOfRules):
    return [rule[aller] for rule in listOfRules if aller in rule.keys()]

In [4]:
from functools import reduce
def getIngredientNoAllerg(aller, listOfRules):
    temp = getIngredientsListForAllerg(aller, listOfRules)
    if(len(temp)>1):
        return {
            'no': reduce(lambda a,b: list(set(a) - set(b)), temp), 
            'oneOf': reduce(lambda a,b: list(set(a).intersection(set(b))), temp)
            }
    return {'oneOf': temp[0]}

def findAlle(allsFound, possi):
    for al in possi:
        if(not(al in allsFound.keys()) and len(possi[al]['oneOf']) == 1):
            return al, possi[al]['oneOf'][0]

def updateData(data, al, ing):
    for alKey in data.keys():
        if(alKey != al and ing in data[alKey]['oneOf']): data[alKey]['oneOf'].remove(ing)
    return data

In [5]:
def findIngWithAlergene(allers, listOfRules):
    toto = {aller:getIngredientNoAllerg(aller, listOfRules) for aller in allers}
    allsFound = {}
    while(len([1 for al in toto.keys() if len(toto[al]['oneOf']) > 1]) > 0):
        al, ing = findAlle(allsFound, toto)
        allsFound[al] = ing
        toto = updateData(toto, al, ing)
    for al in allers:
        if(al not in allsFound.keys()): allsFound[al] = toto[al]['oneOf'][0]
    return allsFound

def findSafeIngr(receipes, als):
    return reduce(lambda a,b: list(set(a+b)),[list(set(rule['ingredients']) - set(als)) for rule in receipes])


In [6]:
def countIngInList(ing, receipes):
    return sum([len(set(rule['ingredients']).intersection(ing)) for rule in receipes])

def getResultForPart2(ingWithAl):
    return reduce(lambda a,b: a+','+ingWithAl[b],sorted(ingWithAl)[1:],ingWithAl[sorted(ingWithAl)[0]])

In [7]:
ingWithoutAllergenes = findSafeIngr(dataInput, list(findIngWithAlergene(listOfAlle,dataInput1).values()))
part2 = findIngWithAlergene(listOfAlle,dataInput1)

print('Part 1 :', countIngInList(ingWithoutAllergenes, dataInput))
print('Part 2 :', getResultForPart2(part2))

Part 1 : 2203
Part 2 : fqfm,kxjttzg,ldm,mnzbc,zjmdst,ndvrq,fkjmz,kjkrm
